# **Finetuning efficientnetv2 **

****Referred from notebooks of [Abhay](https://www.kaggle.com/abhaymishra3420/xception-from-scratch) and [Debarshi](https://www.kaggle.com/debarshichanda/efficientnetv2-state-of-the-art)****

![Efficientnetv2](https://serge-m.github.io/media/2020-07-01-which-backbone-to-choose/efficientnet.png)

In [ ]:
!pip install torchsummary
!pip install timm
!pip install loguru
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torch.nn.functional as F
from torchsummary import summary
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import os
import gc
import copy
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
from loguru import logger

In [ ]:
class CONFIG:
    seed = 42
    model_name = 'tf_efficientnetv2_m_in21k' 
    train_batch_size = 8
    valid_batch_size = 32
    img_size = 150
    epochs = 5
    learning_rate = 1e-4
    min_lr = 1e-6
    weight_decay = 1e-6
    T_max = 5
    scheduler = 'CosineAnnealingLR'
    n_accumulate = 1
    n_fold = 5
    target_size = 270
    device = torch.device( "cpu")

In [ ]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG.seed)

In [ ]:
transform = transforms.Compose([transforms.Resize(CONFIG.img_size),
                                transforms.ToTensor()])

train_dataset = dataset.ImageFolder(root='../input/100-bird-species/train',transform=transform)

test_dataset = dataset.ImageFolder(root='../input/100-bird-species/test',
                                            transform=transform)

In [ ]:
total_class = train_dataset.classes
train_class = train_dataset.class_to_idx
test_class = test_dataset.class_to_idx

idx_to_classes = {}
for classes in train_class:
    idx_to_classes[train_class[classes]] = classes

In [ ]:
train_loader = DataLoader(train_dataset,
                        batch_size=CONFIG.train_batch_size,
                        shuffle=True)

valid_loader = DataLoader(test_dataset,
                        batch_size= CONFIG.valid_batch_size,
                        shuffle=False)

In [ ]:
class BirdModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(BirdModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        for param in self.model.parameters():
            param.requires_grad = False
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(self.n_features, CONFIG.target_size)
        

    def forward(self, x):
        output = self.model(x)
        return output
    
model = BirdModel(CONFIG.model_name,pretrained=False)
model.to(CONFIG.device)
model.load_state_dict(torch.load("../input/trained-effnetv2/epoch3.pb"))

In [ ]:
params_to_update = model.parameters()
feature_extract = True
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

In [ ]:
def criterion(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets.long())

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:         
        images = images.to(device)
        labels = labels.to(device)
        
        batch_size = images.size(0)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss = loss / CONFIG.n_accumulate
            
        loss.backward()
        
        if (step + 1) % CONFIG.n_accumulate == 0:
            optimizer.step()

            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss/dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    TARGETS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:        
        images = images.to(device)
        labels = labels.to(device)
        
        batch_size = images.size(0)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss/dataset_size
        
        PREDS.append(outputs.sigmoid().cpu().detach().numpy())
        TARGETS.append(labels.cpu().detach().numpy())
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    TARGETS = np.concatenate(TARGETS)
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return epoch_loss, 

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG.T_max, eta_min=CONFIG.min_lr)
    elif CONFIG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CONFIG.T_0, T_mult=1, eta_min=CONFIG.min_lr)
    elif CONFIG.scheduler == None:
        return None
        
    return scheduler
optimizer = optim.Adam(params_to_update, lr=CONFIG.learning_rate, weight_decay=CONFIG.weight_decay)
scheduler = fetch_scheduler(optimizer)


In [ ]:
def run(model, optimizer, scheduler, device, num_epochs):    
    start = time.time()
    
    best_model_wts = model.state_dict()
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG.device, epoch=epoch)
        
        valid_epoch_loss = valid_one_epoch(model, optimizer, scheduler,
                                                            dataloader=valid_loader, 
                                                            device=CONFIG.device, epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(valid_epoch_loss)

        best_model_wts = copy.deepcopy(model.state_dict())
        PATH = "epoch{:.0f}.pb".format(epoch)
        torch.save(model.state_dict(), PATH)
        print("Model Saved")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
# Uncomment to train the model . I have trained for 3 epochs and loaded those weights 
# model, history = run(model, optimizer, scheduler=scheduler, device=CONFIG.device, num_epochs=3)